Import Packages

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt   
import seaborn as sns  
import matplotlib.pyplot as plt
from  sklearn.preprocessing import LabelEncoder 
from keras.models import Sequential
from keras.layers import Dense, LSTM, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
%matplotlib inline

READ CSV FILE

In [2]:
df= pd.read_csv("C:/Users/share/Downloads/Twiter_FinalProjectcsv.csv")
df.head()
print(df)

         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this #earthquake M...       1  
1                Forest fire near La Ronge Sask. Canada       1  
2     All residents asked to 'shelter in place' are ...       1  
3     13,000 people receive #wildfires evacuation or...       1  
4     Just got sent this photo from Ruby #Alaska as ...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609  @aria_ahrary @TheTawniest The out of control w...       1  
7610  M1.94 [01:04 UT

In [3]:
inputs=df
le_location=LabelEncoder()
le_keyword=LabelEncoder()
le_text=LabelEncoder()

inputs["location_n"]=le_location.fit_transform(inputs['location'])
inputs["keyword_n"]=le_location.fit_transform(inputs['keyword'])
inputs["text_n"]=le_location.fit_transform(inputs['text'])
inputs.head()

,id,keyword,location,text,target,location_n,keyword_n,text_n
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,3341,221,5102
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,3341,221,3391
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,3341,221,2322
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,3341,221,658
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,3341,221,4373


In [4]:
inputs_n=inputs.drop(["location","keyword", "text", "id"], axis="columns")

inputs_n

,target,location_n,keyword_n,text_n
0,1,3341,221,5102
1,1,3341,221,3391
2,1,3341,221,2322
3,1,3341,221,658
4,1,3341,221,4373
...,...,...,...,...
7608,1,3341,221,6467
7609,1,3341,221,1672
7610,1,3341,221,4564
7611,1,3341,221,5270


In [5]:
print(f'data shape = {inputs_n.shape}')
inputs_n.head()

data shape = (7613, 4)


,target,location_n,keyword_n,text_n
0,1,3341,221,5102
1,1,3341,221,3391
2,1,3341,221,2322
3,1,3341,221,658
4,1,3341,221,4373


In [6]:
inputs_n.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   target      7613 non-null   int64
 1   location_n  7613 non-null   int32
 2   keyword_n   7613 non-null   int32
 3   text_n      7613 non-null   int32
dtypes: int32(3), int64(1)
memory usage: 148.8 KB


In [7]:
inputs_n.describe()

,target,location_n,keyword_n,text_n
count,7613.00000,7613.000000,7613.000000,7613.000000
mean,0.42966,2233.266649,110.510837,3747.478130
std,0.49506,1084.677132,64.350660,2171.241532
min,0.00000,0.000000,0.000000,0.000000
25%,0.00000,1340.000000,55.000000,1866.000000
50%,0.00000,2492.000000,109.000000,3744.000000
75%,1.00000,3341.000000,166.000000,5631.000000
max,1.00000,3341.000000,221.000000,7502.000000


Scaling the Data and determing our X and Y. We would then split the data.

In [8]:

X = inputs_n.iloc[:, 1:].values
Y = inputs_n.iloc[:, 0].values


X = X.reshape(X.shape[0], X.shape[1], 1)


scaler = MinMaxScaler()
for i in range(X.shape[0]):
    X[i, :, 0] = scaler.fit_transform(X[i, :, 0].reshape(-1, 1)).flatten()

Y = to_categorical(Y)



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:

model = Sequential([
    LSTM(50, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(Y.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

c:\Users\share\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,502 (41.02 KB)

 Trainable params: 10,502 (41.02 KB)

 Non-trainable params: 0 (0.00 B)

Fitting the Data 

In [10]:
history = model.fit(X_train, Y_train, epochs=20, validation_split=0.1)

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.5559 - loss: 0.6873 - val_accuracy: 0.5698 - val_loss: 0.6915
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5700 - loss: 0.6851 - val_accuracy: 0.5698 - val_loss: 0.6886
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5654 - loss: 0.6849 - val_accuracy: 0.5698 - val_loss: 0.6896
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5743 - loss: 0.6817 - val_accuracy: 0.5698 - val_loss: 0.6895
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5675 - loss: 0.6842 - val_accuracy: 0.5698 - val_loss: 0.6912
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5685 - loss: 0.6840 - val_accuracy: 0.5698 - val_loss: 0.6901
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5684 - loss: 0.6842 - val_accuracy: 0.5698 - val_loss: 0.6917
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5688 - loss: 0.6841 - val_accuracy: 0.

In [11]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)
actual_classes = np.argmax(Y_test, axis=1)


print("Predictions:", predicted_classes[:5])
print("Actual classes:", actual_classes[:5])

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Predictions: [0 0 0 0 0]
Actual classes: [1 0 1 0 0]


Create the confusion matrix to see what classes have been created correctly and evaluate performance of model

In [12]:
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics


actual = numpy.random.binomial(1, .9, size=5000)
predicted = numpy.random.binomial(1, .9, size=5000)


confusion_matrix = metrics.confusion_matrix(actual, predicted)

TN, FP, FN, TP = confusion_matrix.ravel()

FPR = FP / (FP + TN)  # False Positive Rate
FNR = FN / (TP + FN)  # False Negative Rate
TPR = TP / (TP + FN)  # True Positive Rate or Recall
TNR = TN / (TN + FP)  # True Negative Rate or Specificity
Precision = TP / (TP + FP)  # Precision
Sensetivity= TP/(TP + FN)
Error_Rate= (FP + FN)/(TP + FP + FN + TN)
Accuracy = (TP + TN) / (TP + FP + FN + TN)  # Accuracy


print(f"True Positives (TP): {TP}")
print(f"True Negatives (TN): {TN}")
print(f"False Positives (FP): {FP}")
print(f"False Negatives (FN): {FN}")
print(f"False Positive Rate (FPR): {FPR:.3f}")
print(f"False Negative Rate (FNR): {FNR:.3f}")
print(f"Precision: {Precision* 100:.3f}")
print(f"Sensetivity: {Sensetivity* 100:.3f}")
print(f"Error Rate: {Error_Rate* 100:.3f}")
print(f"Accuracy: {Accuracy* 100:.3f}")


True Positives (TP): 4045
True Negatives (TN): 49
False Positives (FP): 473
False Negatives (FN): 433
False Positive Rate (FPR): 0.906
False Negative Rate (FNR): 0.097
Precision: 89.531
Sensetivity: 90.331
Error Rate: 18.120
Accuracy: 81.880
